In [3]:
import json
import numpy as np
import faiss
import os


In [4]:
file_path = '../Data/data.json'
with open(file_path, 'r') as json_file:
    data = json.load(json_file)

In [5]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('distilbert-base-nli-mean-tokens')

In [62]:
Emb_dir="../Data/Embeddings"
titles=[]
for Id,product in data.items():
    # print(Id,"->",product['title'])
    # Encode titles into embeddings
    titles.append(product['title'])
    title_embedding = model.encode(product['title'])
    embeddings = np.array(title_embedding)  # Your embeddings here
    # Save embeddings to a .npy file
    np.save(f'../Data/Embeddings/emb{Id}.npy', embeddings)

In [63]:
file_names = os.listdir(Emb_dir)
sorted_file_names = sorted(file_names, key=lambda x: int(x[3:-4]))
saved_embeddings_path=[]
for file in sorted_file_names:
    path=os.path.join(Emb_dir,file)
    saved_embeddings_path.append(path)

In [66]:
# saved_embeddings_path
# titles

In [61]:
all_embeddings = []
# Iterate through each file path
for file_path in saved_embeddings_path:
    # Open the file in binary mode
    with open(file_path, 'rb') as fp:
        # Load the embeddings from the file
        embeddings = np.load(fp)
        # Append the loaded embeddings to the list
        all_embeddings.append(embeddings)

# Concatenate all embeddings in the list along the first axis to form a single array
sentence_embeddings = np.array(all_embeddings)
# sentence_embeddings.shape

In [82]:
#titles --> sentences
#embeddings --> sentence_embeddings
d=sentence_embeddings.shape[1]
index = faiss.IndexFlatL2(d)  # L2 distance is used for similarity search
index.is_trained
index.add(sentence_embeddings)
index.ntotal

78

In [72]:
titles

['Patterns of Preaching: A Sermon Sampler',
 'Candlemas: Feast of Flames',
 'World War II Allied Fighter Planes Trading Cards',
 'Life Application Bible Commentary: 1 and 2 Timothy and Titus',
 'Prayers That Avail Much for Business: Executive',
 'How the Other Half Lives: Studies Among the Tenements of New York',
 'Batik',
 'Losing Matt Shepard',
 'Making Bread: The Taste of Traditional Home-Baking',
 'The Edward Said Reader',
 'Resetting the Clock : Five Anti-Aging Hormones That Improve and Extend Life',
 'Fantastic Food with Splenda : 160 Great Recipes for Meals Low in Sugar, Carbohydrates, Fat, and Calories',
 'Clockwork Worlds : Mechanized Environments in SF (Contributions to the Study of Science Fiction and Fantasy)',
 'Later...',
 'Wake Up and Smell the Coffee',
 'War at Sea: A Naval History of World War II',
 'Telecommunications Cost Management',
 'Sol to Soul',
 'Ultimate Marvel Team-Up',
 'Computed Tomography : Fundamentals, System Technology, Image Quality, Applications',
 'C

In [83]:
query_emb=model.encode(['Music in Western Europe, 1400-1600 (Norton Introduction to Music Hist'])
k=3

In [98]:
# %%time
D,I=index.search(query_emb,3)
print(I)

[[58 67 50]]


In [99]:
titles[50]

"Secrets of Gypsy Love Magick (Llewellyn's New Age Series)"

In [100]:
output=[f'{i}:{titles[i]}'for i in I[0]].copy()
for i in output:
    print(i)

58:Renaissance Music: Music in Western Europe, 1400-1600 (Norton Introduction to Music History)
67:Negotiated Empires: Centers and Peripheries in the New World, 1500-1820
50:Secrets of Gypsy Love Magick (Llewellyn's New Age Series)
